In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install cartopy

In [4]:
#!pip install matplotlib==3.1.3

In [1]:
import os
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras_preprocessing.image import img_to_array
from matplotlib import image
from tqdm import tqdm

In [ ]:
def image_feature(path, direc):
    model = InceptionV3(weights='imagenet', include_top=False)
    features = []
    img_name = []
    # path = "/content/drive/MyDrive/ECRP_Data_Science/"
    for i in tqdm(direc):
        fname = path + 'skt' + '/' + i
        print(fname)
        img = image.load_img(fname)
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feat = model.predict(x)
        feat = feat.flatten()
        features.append(feat)
        img_name.append(i)
    return features, img_name


def image_saving(data_var, saving_path):
    path = saving_path + "/" + data_var._name
    print("creating path:" + path)
    os.makedirs(path, exist_ok=True)
    for index in range(data_var.time.size):
        one_day_variable = data_var.isel(time=index)
        fig = plt.figure(figsize=(161, 201))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())  # adding project within the opened figure (projection=ccrs.LambertConformal() preferred)
        ax.imshow(one_day_variable - 273.5, extent=(
            one_day_variable.longitude.min(), one_day_variable.longitude.max(), one_day_variable.latitude.min(),
            one_day_variable.latitude.max()), cmap='jet', origin='lower')
        plt.xlabel('x')
        plt.ylabel('y')

        # these are additional features you can add to geo plots (like boders, rivers, lakes..etc)
        states_provinces = cf.NaturalEarthFeature(
            category='cultural',
            name='admin_1_states_provinces_lines',
            scale='10m',
            facecolor='none')
        ax.add_feature(cf.BORDERS, edgecolor='blue')
        ax.add_feature(states_provinces, edgecolor='blue')
        ax.add_feature(cf.LAND)
        ax.add_feature(cf.COASTLINE)
        ax.add_feature(cf.OCEAN)
        ax.add_feature(cf.LAKES, alpha=0.5)
        ax.add_feature(cf.RIVERS)

        # adding colorbar and adjust the size
        # cbar = fig.colorbar(mp, shrink=0.4)
        # cbar.minorticks_on()

        # adding the long lat grids and enabling the tick labels
        gl = ax.gridlines(draw_labels=True, alpha=0.5)
        gl.top_labels = True
        gl.right_labels = True

        plt.savefig(path + "/" + data_var._name + "_" + str(index) + ".jpg", dpi=10, bbox_inches='tight')
        plt.clf()
        plt.close(fig)


if __name__ == '__main__':
    data = xr.open_dataset("/content/drive/MyDrive/ECRP_Data_Science/Image_Generation_from_netCDFdata/ERA5_sample_hourly_20200201-20200331.nc")
    print(data.data_vars)
    image_saving(data['skt'], "/content/drive/MyDrive/skt")
    # for data_key in data.data_vars:
    #        image_saving(data[data_key], "/content/drive/MyDrive/ECRP_Data_Science/")


Data variables:
    skt      (time, latitude, longitude) float32 ...
    slhf     (time, latitude, longitude) float32 ...
    sp       (time, latitude, longitude) float32 ...
    sshf     (time, latitude, longitude) float32 ...
    t2m      (time, latitude, longitude) float32 ...
    u10      (time, latitude, longitude) float32 ...
    v10      (time, latitude, longitude) float32 ...
creating path:/content/drive/MyDrive/skt/skt
